In [1]:
# Install libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create string of location
path = '/Users/aegarcia/Documents/Documents/01-2022 Instacart Basket Analysis/'
# Import customers.csv 
df_cust = pd.read_csv(os.path.join(path, '02 Data','Original Data', 'customers.csv'), index_col = False)
# Import Prepared data
df_ords = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_flags.pkl'))

# 1. Wrangle data

In [3]:
# Look at type of data in customer.csv
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 1.1 Rename Columns

In [4]:
# Rename columns. Remove capital letters, use _ for spaces.
df_cust.rename(columns={'First Name':'first_name', 'Surnam':'last_name', 'Gender':'gender', 'STATE':'state', 'Age':'age', 'n_dependants':'num_dependents'}, inplace = True)

In [5]:
# Check new index
df_cust.head()

,user_id,first_name,last_name,gender,state,age,date_joined,num_dependents,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
# Look at type of data in orders_products_flgs.csv
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_label,Busy_days_by_orders,Busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,frequency_of_orders,order_frequency_flag
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,9.0,Mid-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer
1,2539329,1,1,2,8,NaN,14084.0,2.0,0.0,Organic Unsweetened Vanilla Almond Milk,...,12.5,Mid-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer
2,2539329,1,1,2,8,NaN,12427.0,3.0,0.0,Original Beef Jerky,...,4.4,Low-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer
3,2539329,1,1,2,8,NaN,26088.0,4.0,0.0,Aged White Cheddar Popcorn,...,4.7,Low-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer
4,2539329,1,1,2,8,NaN,26405.0,5.0,0.0,XL Pick-A-Size Paper Towel Rolls,...,1.0,Low-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer


# 2. Consistency Checks

In [7]:
# Number of Rows & Columns
df_cust.shape

(206209, 10)

In [8]:
# Numeric data, check if numbers make sense
df_cust.describe()

,user_id,age,num_dependents,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [9]:
# info on every column
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   user_id         206209 non-null  int64 
 1   first_name      194950 non-null  object
 2   last_name       206209 non-null  object
 3   gender          206209 non-null  object
 4   state           206209 non-null  object
 5   age             206209 non-null  int64 
 6   date_joined     206209 non-null  object
 7   num_dependents  206209 non-null  int64 
 8   fam_status      206209 non-null  object
 9   income          206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


### CHECK: 'first_name only has 194, 950 values, is missing 11,259 values

## 2.1 : Mising Values - 'first_name'

In [10]:
df_cust.isnull().sum()

user_id               0
first_name        11259
last_name             0
gender                0
state                 0
age                   0
date_joined           0
num_dependents        0
fam_status            0
income                0
dtype: int64

In [11]:
df_missing_name = df_cust[df_cust['first_name'].isnull() == True]
df_missing_name

,user_id,first_name,last_name,gender,state,age,date_joined,num_dependents,fam_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


### A little over 5% of the cutomers are missing their first name in our database.  I would address this issue to a senior analyst or an engineer. I will delete it for now and we can still refer to our customers as Mr. or Ms. 

In [12]:
# Remove 'first_name' column
df_cust = df_cust.drop(columns = ['first_name'])
df_cust.head()

,user_id,last_name,gender,state,age,date_joined,num_dependents,fam_status,income
0,26711,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 2.2: Mixed data types 

In [13]:
# Mixed data types
for col in df_cust.columns.tolist():
  mixed = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[mixed]) > 0:
    print (col)

### No mixed data type columns! 

## 2.3: Duplicates

In [14]:
df_dup = df_cust[df_cust.duplicated()]
df_dup

,user_id,last_name,gender,state,age,date_joined,num_dependents,fam_status,income


### No duplicates found!

# 3. Merge data

In [15]:
df_cust.head()

,user_id,last_name,gender,state,age,date_joined,num_dependents,fam_status,income
0,26711,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [16]:
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_label,Busy_days_by_orders,Busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,frequency_of_orders,order_frequency_flag
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,9.0,Mid-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer
1,2539329,1,1,2,8,NaN,14084.0,2.0,0.0,Organic Unsweetened Vanilla Almond Milk,...,12.5,Mid-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer
2,2539329,1,1,2,8,NaN,12427.0,3.0,0.0,Original Beef Jerky,...,4.4,Low-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer
3,2539329,1,1,2,8,NaN,26088.0,4.0,0.0,Aged White Cheddar Popcorn,...,4.7,Low-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer
4,2539329,1,1,2,8,NaN,26405.0,5.0,0.0,XL Pick-A-Size Paper Towel Rolls,...,1.0,Low-range product,Regularly days,Average orders,11,Regular customer,6.367797,Low spender,20.0,Regular customer


### Will use the 'user_id as the common column in both dataframes

In [17]:
# Merge customer data with Ords_prods
df_ords_prods_cust = df_ords.merge(df_cust, on = 'user_id', how = 'left', indicator = True)

In [18]:
df_ords_prods_cust.shape

(32641268, 31)

In [19]:
df_ords_prods_cust['_merge'].value_counts()

both          32641268
right_only           0
left_only            0
Name: _merge, dtype: int64

In [20]:
df_ords_prods_cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_time,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,order_frequency_flag,last_name,gender,state,age,date_joined,num_dependents,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196.0,1.0,0.0,Soda,...,Regular customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2539329,1,1,2,8,NaN,14084.0,2.0,0.0,Organic Unsweetened Vanilla Almond Milk,...,Regular customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2539329,1,1,2,8,NaN,12427.0,3.0,0.0,Original Beef Jerky,...,Regular customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2539329,1,1,2,8,NaN,26088.0,4.0,0.0,Aged White Cheddar Popcorn,...,Regular customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,2539329,1,1,2,8,NaN,26405.0,5.0,0.0,XL Pick-A-Size Paper Towel Rolls,...,Regular customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


# Export data

In [21]:
df_cust.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'customers_revised.pkl'))
df_ords_prods_cust.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers.pkl'))